### Preparing assembly files for binning
To create a depth processing file, reads must be re-aligned to the contigs, or mapping. This has been done using bowtie2 (can also be done using BWA). The next step would be to create a depth file with MetaBat2, convert that to be suitable for CONCOCT and MaxBin2, and then process these into bins. 

This is all assuming you have installed all of the softwares mentioned here. Use conda for quick install. If needed, the documentation for everything can be found here:

Metabinner: https://github.com/ziyewang/MetaBinner

MetaBAT: https://bitbucket.org/berkeleylab/metabat/src/master/README.md

CONCOCT: https://github.com/BinPro/CONCOCT

CheckM: https://github.com/Ecogenomics/CheckM/wiki

Das_tool: https://github.com/cmks/DAS_Tool

#### MetaBat2
The first piece of code here generates a fairly simple text file for the coverage of these files. The next set of code runs MetaBat2  (v2.10.2) using minContig 1500, minCV 1.0, minCVSum 1.0, maxP 95%, minS 60, lcuster size 50000 and maxEdges 200. It sets the minimum size for a bin to 200000 basepairs, which is fairly low, so you can keep it. It gathers all mapping information into a single depth file, so you can use your 1 file in the next analysis. An important parameter to play around with is the minimum bin size. When set to 200000, this will severely limit the amount of bins you gain, especially if your samples aren't perfect. Therefore, it is wise to run MetaBAT several times with slight alterations to the -s flag to find your optimal setting (you don't want 3 bins, you also don't want 1000).

For a reference on how to do this accurately, use: https://bitbucket.org/berkeleylab/metabat/wiki/Best%20Binning%20Practices

In [ ]:
#set parameters for binning:
SAMPLENAME=nameofsample
CONTIGFILE=nameofcontigfile.fa
OUTDIR=MetaBAT_"$SAMPLENAME"_bins
MAPPINGPATH=/path/to/mappingfiles/
CONTIGPATH=/path/to/contigs/
CHECKMPATH=/path/to/checkm/database/

[ -d $MAPPINGPATH ] || { echo 'Invalid path to mapping files, exiting' && exit; }
[ -d $CONTIGPATH ] || { echo 'Invalid path to contigs, exiting' && exit; }
[ -d $CHECKMPATH ] || { echo 'Invalid path to contigs, exiting' && exit; }

#this creates a depth file for MetaBat2
jgi_summarize_bam_contig_depths --outputDepth ../data/working/MetaBAT_"$SAMPLENAME"_depth.txt $MAPPINGPATH/*.bam || { echo 'Exit code 1: Failed to create depth file, exiting' && exit; }

#this is the actual MetaBat2 script with verbose output, minimum length 1500 and bin size 50000

mkdir ../data/results/$OUTDIR

metabat2 -i $CONTIGPATH/$CONTIGFILE -a ../data/working/MetaBAT_"$SAMPLENAME"_depth.txt \
-o ../data/results/$OUTDIR/Metabin \
-m 1500 \
-s 50000

#this runs CheckM immediately after and puts the results alongside your bins
export CHECKM_DATA_PATH=$CHECKMPATH
checkm data setRoot $CHECKMPATH
checkm lineage_wf -x fa -t $NSLOTS ../data/results/$OUTDIR ../data/results/$OUTDIR/bins-stats || { echo 'Exit code 3: CheckM failed' && exit; }

 #### CONCOCT
 This set of commands runs CONCOCT in its standard mode. It first creates a depth/coverage file for itself to use and then runs CONCOCT, with the standard settings. This means k-mer value is set to 4, minimum contig length is 1000, and CONCOCT runs on the exact amount of slots given to it by Hydra. 
 
CONCOCT creates a depth file out of the coverance created in the mapping step. It is key that this is all in the correct places before proceeding with binning. It creates a single file, which is then used for the complete binning process. Do keep in mind that binning might take awhile, so be prepared to let this run overnight.

IMPORTANT: in the current version of CONCOCT, you're missing a vital file, called libmkl.so. Without this file, CONCOCT will not be able to start. You can fix this issue by installing another file through Conda: 

conda install mkl

Additionally, samtools will not work properly after a fresh CONCOCT install. The easiest way to fix this is to go to your environment where you installed CONCOCT and force an update through conda. 

In [ ]:
#this creates the CONCOCT depth file
SAMPLENAME=nameofsample
CONTIGFILE=nameofcontigfile
OUTDIR=CONCOCT_"$SAMPLENAME"_bins
TEMPDIR=CONCOCT_"$SAMPLENAME"_temp
MAPPINGPATH=/path/to/mappingfiles/
CONTIGPATH=/path/to/contigs/
CHECKMPATH=/path/to/checkm/database/

[ -d $MAPPINGPATH ] || { echo 'Invalid path to mapping files, exiting' && exit; }
[ -d $CONTIGPATH ] || { echo 'Invalid path to contigs, exiting' && exit; }
[ -d $CHECKMPATH ] || { echo 'Invalid path to contigs, exiting' && exit; }
#this part cuts up the contigs into 10kb pieces for CONCOCT to use 
cut_up_fasta.py $CONTIGPATH/$CONTIGFILE -c 10000 -o 0 --merge_last -b ../data/working/$SAMPLENAME_contigs_cut.bed > ../data/working/$SAMPLENAME_contigs_cut.fa || { echo 'Exit code 1: failed to cut up contigs, exiting.' && exit; }

#this part estimates contig coverage
concoct_coverage_table.py ../data/working/$SAMPLENAME_contigs_cut.bed $MAPPINGPATH/*.bam > ../data/working/coverage_table_$SAMPLENAME.tsv || { echo 'Exit code 2: failed to create coverage file, exiting.' && exit; }

#CONCOCT script
mkdir ../data/results/$OUTDIR
mkdir ../data/working/$TEMPDIR

#this next bit actually runs CONCOCT itself
concoct --composition_file ../data/working/$SAMPLENAME_contigs_cut.fa --coverage_file ../data/working/coverage_table_$SAMPLENAME.tsv -t $NSLOTS -b ../data/working/$TEMPDIR || { echo 'Exit code 3: CONCOCT failed to run, exiting.' && exit; }
merge_cutup_clustering.py ../data/working/$TEMPDIR/clustering_gt1000.csv > ../data/working/$TEMPDIR/$SAMPLENAME_clustering_merged.csv || { echo 'Exit code 4: failed to merge clusters, exiting.' && exit; }
extract_fasta_bins.py $CONTIGPATH/$CONFIGFILE ../data/working/$TEMPDIR/$SAMPLENAME_clustering_merged.csv --output_path ../data/results/$OUTDIR || { echo 'Exit code 5: Bins were not extracted, exiting.' && exit; }

#this runs CheckM immediately after and puts the results alongside your bins
export CHECKM_DATA_PATH=$CHECKMPATH
checkm data setRoot $CHECKMPATH
checkm lineage_wf -x fa -t $NSLOTS ../data/results/$OUTDIR ../data/results/$OUTDIR/bins-stats || { echo 'Exit code 6: CheckM failed' && exit; }

You can now proceed to actually running Metabinner. 

### COCACOLA
COCACOLA is a four-way hybrid automated binning software that relies on some of the functionalities that CONCOCT has. **Under construction, do not use**

In [ ]:
COCACOLA=~/COCACOLA/cocacola.py
CONTIGS=/path/to/fixed/contigs/
COMPOSITION
ABUNDANCE=/path/to/
cocacola.py [-h] [--contig_file CONTIG_FILE] [--abundance_profiles ABUNDANCE_PROFILES] [--composition_profiles COMPOSITION_PROFILES] [--edge_list EDGE_LIST] [--output OUTPUT] [--clusters CLUSTERS]

### Bin_refiner
This runs an ensemble binning software on your bins. There are two options here: one is to run all of the bins together and allow the software to choose a better set for you. Alternatively, you can use the second script, below the first one, to make different combinations of the three and then choose which two sets to combine into your perfect set of bins.

In [ ]:
SAMPLENAME=
CONCOCTBINS=/path/to/CONCOCT/bins
MetaBATBINS=/path/to/MetaBAT/bins
EXTRABINNERBINS=/path/to/extra/binner/bins/
INDIR=Refiner_"$SAMPLENAME"_bins
OUTDIR=Final_bins_"$SAMPLENAME"
CHECKMPATH=/path/to/CHECKM/DATABASE

mkdir ../data/working/$INDIR
mkdir ../data/working/$INDIR/CONCOCT
mkdir ../data/working/$INDIR/MetaBAT
mkdir ../data/working/$INDIR/Extrabinner
mkdir ../data/results/$OUTDIR

cp $CONCOCTBINS/*.fa ../data/working/$INDIR/CONCOCT/
cp $MetaBATBINS/*.fa ../data/working/$INDIR/MetaBAT/
cp $EXTRABINNERBINS/*.fa ../data/working/$INDIR/Extrabinner/

INPUT=../data/working/$INDIR/
Binning_refiner -i $INPUT -p $SAMPLENAME -plot -m 50

mv "$SAMPLENAME"_Binning_refiner_outputs/"$SAMPLENAME"_refined_bins/*.fasta ../data/results/$OUTDIR
#run CheckM for quick assessment
export CHECKM_DATA_PATH=$CHECKMPATH
checkm data setRoot $CHECKMPATH
checkm lineage_wf -x fasta -t $NSLOTS ../data/results/$OUTDIR ../data/results/$OUTDIR/bins-stats || { echo 'Exit code 5: CheckM failed' && exit; }

#this next bit runs busco on your final bins

conda deactivate
conda activate busco
busco -m  genome -i ../data/results/$OUTDIR -o ../data/results/$OUTDIR/bins-stats-Busco --auto-lineage -c $NSLOTS --download-path $DOWNLOADPATH || { echo 'Exit code 6: BUSCO failed, exiting.' && exit; }

#Then we generate files for Anvi'O processing:
cd ../data/results/$OUTDIR/
#the next loop generates a txt file which contains all the necessary information for Anvi'O
for fasta_file in *.fasta
do
    bin_name=`echo $fasta_file | awk 'BEGIN{FS="."}{print $1}'`;
    for contig_id in `grep '>' $fasta_file | sed 's/\>//g'`
    do
        echo -e "$contig_id\t$bin_name"
    done
done > collection.txt
#this removes the > so Anvi'O recognises the collection and labels it correctly
sed 's/>/ /' collection.txt > "$SAMPLENAME"_collection.txt
#remove the old collection to prevent confusion
rm collection.txt

### Continuing and troubleshooting
You should now have 3 sets of bins, each created with a slightly different algorithm, consolidated into a single set of bins through DAS_tools. It is now important to run the CheckM software with the script below and generate output files for all of them. This will inform you towards the quality of your bins and your contamination/completion rate. After this, you can proceed to the "Refine Bins" part of the workflow.

CheckM runs a check against a database to determine the levels of completeness versus contamination. These statistics are vital in determining how you want to proceed in the refinement process. Mind you, CheckM works without setting the database you need, but you get very confusing data. So make sure you set it correctly before running it. The scripts above run CheckM intrinsically, but its good to know that CheckM is the reason these scripts need to be run on a himem node (it regularly spikes above the 16G of RAM used per node, so yeah). 

MetaBAT is extremely annoying in the fact that it won't create its own directories. Make sure the directories are in place before it actually runs. 

CONCOCT will in general create more bins than MetaBAT2, but you can quite likely discard quite a few since they're going to be 3000 bp long which is not a lot (although it could be a viral sequence).

Congratulations! You have finished binning. The bins you have produced are considered putative genomes and can be used for a fair amount of practices, some of which I have listed in the Anvi'O notebook, others which are in the Analysis notebook. Good luck!